In [47]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import json
import matplotlib.pyplot as plt


api_key = "REDACTED" 

df = pd.DataFrame()

In [51]:
# Function to easily call the api for various cities
def call(func_df, city):

    # params = {
    #     "lat": lat,
    #     "lon": lon,
    #     "appid": api_key,
    #     "units": "imperial"   # metric, imperial, or standard
    # }
    func_df['city'] = city
    
    if city in func_df['city'].unique():
        return "City already in DF"

    
    params = {
        "q": city,
        "appid": api_key,
        "units": "imperial"   # metric, imperial, or standard
    }

    # URL for api_call
    url = "https://api.openweathermap.org/data/2.5/forecast"

    
    # API Call
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()

    # Convert to JSON
    data = response.json()

    # Create DF
    func_df = pd.json_normalize(data["list"], sep="_")

    # Flatten meta data in weather dict
    df_weather = pd.json_normalize(func_df['weather'].str[0]).add_prefix("weather_")
    func_df = func_df.drop(columns=['weather']).join(df_weather)

    # Convert to datetime
    func_df['dt_txt'] = pd.to_datetime(func_df['dt_txt'])

    return func_df

In [53]:
# call API call function for specified lat and lon

lat = 40.7128
lon = -74.0060
city = "Chicago"

df = pd.concat([df, call(df, city)], ignore_index=True)

In [55]:
print(df)

   city            dt  visibility   pop              dt_txt  main_temp  \
0   NaN  1.771632e+09     10000.0  0.00 2026-02-21 00:00:00      32.34   
1   NaN  1.771643e+09     10000.0  0.00 2026-02-21 03:00:00      29.66   
2   NaN  1.771654e+09     10000.0  0.00 2026-02-21 06:00:00      26.24   
3   NaN  1.771664e+09     10000.0  0.00 2026-02-21 09:00:00      23.90   
4   NaN  1.771675e+09     10000.0  0.00 2026-02-21 12:00:00      22.05   
5   NaN  1.771686e+09     10000.0  0.00 2026-02-21 15:00:00      30.34   
6   NaN  1.771697e+09     10000.0  0.00 2026-02-21 18:00:00      33.62   
7   NaN  1.771708e+09     10000.0  0.00 2026-02-21 21:00:00      33.69   
8   NaN  1.771718e+09     10000.0  0.00 2026-02-22 00:00:00      32.05   
9   NaN  1.771729e+09     10000.0  0.00 2026-02-22 03:00:00      29.35   
10  NaN  1.771740e+09       753.0  0.20 2026-02-22 06:00:00      27.10   
11  NaN  1.771751e+09       777.0  1.00 2026-02-22 09:00:00      25.93   
12  NaN  1.771762e+09      4017.0  1.0

In [ ]:
df.plot(x='dt_txt', y='main_temp')
df.show()

In [ ]:
# Dictiponary comperhension (USE THIS!) for look to check out what it's in items.
filtered = {k:v for k, v in data.items() if k != "list"}

city_data = data["city"]

print(city_data)

In [ ]:
# Convert weather to category codes
df["dt_txt"] = pd.to_datetime(df["dt_txt"])
df["main_temp"] = pd.to_numeric(df["main_temp"], errors="coerce")
df = df.sort_values("dt_txt")

changed = df["weather_main"].ne(df["weather_main"].shift())

fig, ax = plt.subplots()
ax.plot(df["dt_txt"], df["main_temp"])
ax.set_xlabel("Time")
ax.set_ylabel("Temperature")

ymin, ymax = ax.get_ylim()

for t, state in df.loc[changed, ["dt_txt", "weather_main"]].itertuples(index=False):
    ax.axvline(t, linestyle="--", alpha=0.4)
    ax.text(t, ymax, f" {state}", rotation=90, va="top", ha="left")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()